In [4]:
import os
from nbpapaya import Brain

### Data without motion correction

In [19]:
Brain([os.path.abspath("data/rest.nii.gz"),os.path.abspath("data/rest_DMN.nii.gz")],port=8888)

doing checks /home/caroline/.jupyter/custom/


#  6 parameter model motion correction

In [22]:
Brain([os.path.abspath("data/rest_6mot.nii.gz"),os.path.abspath("data/rest_6mot_DMN.nii.gz")],port=8888)

doing checks /home/caroline/.jupyter/custom/


# Friston 24 Parameter Model Motion Correction

In [18]:
Brain([os.path.abspath("data/rest_24mot.nii.gz"),os.path.abspath("data/rest_24mot_DMN.nii.gz")],port=8888)

doing checks /home/caroline/.jupyter/custom/
